# 🧠 Atlas — Interactive Demo

Welcome! This notebook walks you through the Atlas long-form memory system step by step.

**What you'll learn:**
1. How the agent extracts memories from conversation
2. How memories are stored and retrieved
3. How to test memory recall
4. How to use different providers (Ollama, Gemini, Groq)

> 📊 For full benchmarks and evaluation results, see **[results.ipynb](results.ipynb)**

---

## 1. Setup

First, install dependencies and load the environment.

In [ ]:
import os
import sys
import json
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.agent import LongMemAgent
from dotenv import load_dotenv

load_dotenv()

print('✓ Dependencies loaded')
print(f'  Project root: {project_root}')

## 2. Choose Your Provider

Atlas supports multiple LLM backends. Pick one:

| Provider | Model | API Key Needed | Notes |
|----------|-------|:--------------:|-------|
| `ollama` | `mistral` | No | Local, fast, free. Run `ollama serve` first |
| `gemini` | `gemma-3-27b-it` | `GEMINI_API_KEY` | Free tier, 30 req/min |
| `groq` | `llama-3.1-8b-instant` | `GROQ_API_KEY` | Very fast, rate limited |

Edit the cell below to change providers.

In [ ]:
# ═══════════════════════════════════════════
# 🔧 CONFIGURE YOUR PROVIDER HERE
# ═══════════════════════════════════════════

PROVIDER = 'ollama'                 # 'ollama', 'gemini', or 'groq'
MODEL    = 'mistral'                # model name for your provider
# MODEL  = 'gemma-3-27b-it'         # uncomment for Gemini
# MODEL  = 'llama-3.1-8b-instant'   # uncomment for Groq

# ═══════════════════════════════════════════

# Clean up any previous demo database
demo_db = 'demo_interactive.db'
if os.path.exists(demo_db):
    os.remove(demo_db)
    print(f'🗑️  Cleaned previous {demo_db}')

# Initialize agent
agent = LongMemAgent(
    provider=PROVIDER,
    model=MODEL,
    db_path=demo_db,
    context_limit=4096,
    flush_threshold=0.70
)

print(f'\n✓ Agent initialized')
print(f'  Provider: {agent.provider}')
print(f'  Model:    {agent.model}')
print(f'  DB:       {demo_db}')

## 3. Chat Helper

A nice helper to display conversations with memory metadata.

In [ ]:
from IPython.display import display, Markdown, HTML

def chat(message: str, show_memories=True):
    """Send a message and display the response with memory info."""
    print(f'\n👤 You: {message}')
    
    response = agent.chat(message)
    
    print(f'🤖 Atlas: {response["response"]}')
    print(f'\n   ┌─ Turn {response["turn_id"]} │ '
          f'Context: {response["context_utilization"]} │ '
          f'Memories: {response["total_memories"]} │ '
          f'Flush: {"yes" if response["flush_triggered"] else "no"}')
    
    if show_memories and response['active_memories']:
        print(f'   │ 🧠 Retrieved {len(response["active_memories"])} memories:')
        for mem in response['active_memories']:
            print(f'   │   • {mem["content"]} '
                  f'(from turn {mem["origin_turn"]}, conf: {mem["confidence"]:.2f})')
    print('   └──────────────────────────')
    
    return response

print('✓ Chat helper ready. Use chat("your message") below!')

## 4. Plant Some Memories

Let's tell the agent some facts about ourselves. These will be extracted as memories.

In [ ]:
chat('Hi! My name is Jordan and I work as a data scientist in Seattle.')
chat("I'm allergic to peanuts and I follow a vegetarian diet.")
chat('My favorite programming language is Python. I also use R for statistics.')
chat('I have a daughter named Maya, she is 5 years old.')

## 5. Trigger Memory Distillation

The agent normally distills memories when context hits 70% capacity.
We can also trigger it manually to extract memories right now.

In [ ]:
print('⚡ Triggering memory distillation...\n')

result = agent.manual_distill()

print(f'✓ {result["message"]}')
print(f'  Memories added: {result["memories_added"]}')
print(f'  Total memories: {result["total_memories"]}')

## 6. Inspect Stored Memories

Let's see what the distiller extracted.

In [ ]:
memories = agent.get_all_memories()

if memories:
    print(f'📦 Stored Memories ({len(memories)} total):\n')
    for m in memories:
        print(f"  [{m['type']:>12}] {m['key']}: {m['value']}")
        print(f"               confidence: {m['confidence']:.2f}, from turn {m['source_turn']}")
else:
    print('No memories stored yet. Run distillation first.')

## 7. Test Memory Recall

Now the key test — does the agent remember what we told it?

In [ ]:
chat("What's my name?")

In [ ]:
chat('Suggest a dinner recipe for me.')

In [ ]:
chat('What should I get my kid for her birthday?')

In [ ]:
chat('Tell me everything you remember about me.')

## 8. Try It Yourself!

Use the cell below to have your own conversation with the agent.
Each message is processed through the full memory pipeline.

In [ ]:
# Type your message here:
chat('What do I do for work?')

In [ ]:
# Try another:
chat('Your message here')

## 9. Database Peek

Under the hood, memories are stored in SQLite with vector embeddings + full-text search indexes.

In [ ]:
import sqlite3

conn = sqlite3.connect(demo_db)
cursor = conn.cursor()

cursor.execute('SELECT COUNT(*) FROM memories WHERE is_active = 1')
active = cursor.fetchone()[0]

cursor.execute('SELECT COUNT(*) FROM turns')
turns = cursor.fetchone()[0]

cursor.execute('''
    SELECT type, COUNT(*) as count 
    FROM memories WHERE is_active = 1 
    GROUP BY type
''')
types = cursor.fetchall()

print(f'📋 Database: {demo_db}')
print(f'   Active memories: {active}')
print(f'   Total turns: {turns}')
print(f'   Memory types:')
for type_name, count in types:
    print(f'     {type_name}: {count}')

conn.close()

---

## Next Steps

| What | How |
|:-----|:----|
| 📊 **See benchmarks** | Open [results.ipynb](results.ipynb) |
| 💬 **Interactive CLI** | `uv run python main.py` |
| 🧪 **Run evaluation** | `uv run python eval/evaluate.py --provider gemini` |
| 🏠 **Local eval** | `uv run python eval/evaluate.py --local --model mistral` |
| 📖 **Quick eval** | `uv run python eval/evaluate.py --quick --local` |

### Architecture

```
User Message
    │
    ▼
┌─────────────────────────┐
│    Context Manager      │ ← checks if context needs flushing
│    (token counting)     │
└────────┬────────────────┘
         │ if >70% full
         ▼
┌─────────────────────────┐
│    Memory Distiller     │ ← LLM extracts memories from conversation
│    (Groq/Gemini/Ollama) │
└────────┬────────────────┘
         │
         ▼
┌─────────────────────────┐
│    Memory Store         │ ← SQLite + sqlite-vec + FTS5
│    (persist & search)   │
└────────┬────────────────┘
         │
         ▼
┌─────────────────────────┐
│    Memory Retriever     │ ← hybrid vector + keyword search
│    (RRF fusion)         │
└────────┬────────────────┘
         │
         ▼
┌─────────────────────────┐
│    System Prompt        │ ← memories injected into context
│    + LLM Response       │
└─────────────────────────┘
```

## Cleanup (Optional)

In [ ]:
# Uncomment to delete the demo database:
# os.remove(demo_db)
# print(f'✓ Removed {demo_db}')